In [41]:
import pymongo
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint

In [46]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('Backend Final Project-credential.json', scope)
client = gspread.authorize(creds)

sheet = client.open('I-House Mastersheet').get_worksheet(0)

allData = sheet.get_all_records()
for item in allData:
    pprint(item)

{'Dinner: Please check all the items that you would like in your meal box. ': '{Baked '
                                                                              'potato '
                                                                              '(V/GF)=false, '
                                                                              'Dessert '
                                                                              'if '
                                                                              'available=false, '
                                                                              'Walnut '
                                                                              'prawns=false, '
                                                                              'Quinoa '
                                                                              'vegetable '
                                                                              'soup '
          

In [47]:
myclient = pymongo.MongoClient("mongodb://mongo:27017/")
mydb = myclient["tasks"]
mycol = mydb["meals"]

In [48]:
def meal_bool_transform(value):
    if value:
        meal_dict = {}
        value = value[1:-1]

        menu_items = value.split(', ')
        for item in menu_items:
            dict_components = item.split('=')
            meal_dict[dict_components[0]] = (dict_components[1]=='true')
        return meal_dict
    else:
        return value

In [49]:
for record in allData:
    

    meal_count = 0
    lunch_menu = {}
    dinner_menu = {}
    menu = [lunch_menu, dinner_menu]
    
    # remove key and value with '.' and create a new key without '.' with the same value
    key_with_dot = {}
    for key, value in record.items():
        if "." in key:                
            key_with_dot[key] = value
           
            
            

            
    for key, value in key_with_dot.items():
        del record[key]
        new_key = key.replace(".", "")
        if ("Lunch" in key) or ("Dinner" in key):
            new_value = meal_bool_transform(value)
        else:
            new_value = value 
        record[new_key] = new_value           
    
    record["is_completed"] = False
    mycol.insert_one(record)


    

{'Broccoli (V/GF)': False, "Chef's salad": False, 'Pizza': False, 'Coconut rice (V/GF)': False, 'Dessert if available': False, 'Thai coconut curry (V/GF)': False, 'Jerk chicken (GF)': False, 'Chow mein (Vt)': False, 'Korean pork chops': False, 'Pasta': False, 'Cream of tomato soup (Vt)': False}
{'Baked potato (V/GF)': False, 'Dessert if available': False, 'Walnut prawns': False, 'Quinoa vegetable soup (V/GF)': False, 'Onion rings (Vt)': False, 'Roasted vegetables (V/GF)': False}
{'Jerk chicken (GF)': False, 'Chow mein (Vt)': False, 'Korean pork chops': False, 'Pasta': False, 'Cream of tomato soup (Vt)': False, 'Broccoli (V/GF)': False, "Chef's salad": True, 'Pizza': True, 'Coconut rice (V/GF)': False, 'Dessert if available': False, 'Thai coconut curry (V/GF)': False}
{'Baked potato (V/GF)': False, 'Dessert if available': True, 'Walnut prawns': False, 'Quinoa vegetable soup (V/GF)': True, 'Onion rings (Vt)': False, 'Roasted vegetables (V/GF)': True}
{" Chef's salad": True, 'Chow mein (V

In [50]:
for item in mydb.meal_order.find():
    pprint(item)